In [1]:
from selenium import webdriver
import chromedriver_binary

In [3]:
import requests
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
driver.get("https://www.themoviedb.org/list/6564?language=en-US")

In [4]:
soup = BeautifulSoup(driver.page_source,"html.parser")

In [5]:
movies = []
import re
import numpy as np

for movie in soup.find('ul',class_="items_wrapper list_results").find_all('li'):
    #print(movie)
    try:
        title = movie.find('img').get('alt')
        movie_link = "https://www.themoviedb.org/"+movie.find('a').get('href')[1:]
        movie_id = re.split(r'[/?]', movie_link)[4]
        movies.append([title,movie_link,movie_id])
    except:
        title = np.nan
        movie_link = np.nan
        movie_id = np.nan
        
import pandas as pd
data = pd.DataFrame(movies,columns=["title","movie_link","movie_id"])

In [6]:
data.head()

,title,movie_link,movie_id
0,Ek Niranjan,https://www.themoviedb.org/movie/38220?languag...,38220
1,Bāhubali: The Beginning,https://www.themoviedb.org/movie/256040?langua...,256040
2,Mirchi,https://www.themoviedb.org/movie/176590?langua...,176590
3,Rebel,https://www.themoviedb.org/movie/135870?langua...,135870
4,Mr. Perfect,https://www.themoviedb.org/movie/63376?languag...,63376


In [7]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'a1c7c41d7acd5a3c303c90d19b72ebe4'

In [8]:
from tmdbv3api import Movie
tmdb_movie = Movie()

def getMovieDetails(movieID):
    details = []
    response1 = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movieID,tmdb.api_key))
    response2 = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movieID,tmdb.api_key))
    resp1_json = response1.json()
    resp2_json = response2.json()
    
    genres = []
    for genre in resp1_json['genres']:
        genres.append(genre['name'])
    genres = ",".join(genres)
        
    overview = resp1_json['overview'].replace(" ",",")
    
    casting = []
    for i in range(0,3):
        if resp2_json['cast'][i]['name']:
            casting.append(resp2_json['cast'][i]['name'].replace(" ",''))
        else:
            break
    
    crew=""
    for role in resp2_json['crew']:
        if role['job'] == 'Director':
            crew = role['name']
            break
    crew = crew.replace(" ","")

    return [genres, overview, casting, crew]


new_columns = data['movie_id'].apply(getMovieDetails)
new_columns_df = pd.DataFrame(new_columns.tolist(), columns=['genres', 'overview', 'cast', 'crew'])
data2 = pd.concat([data, new_columns_df], axis=1)

In [9]:
data2['genres'] = data2['genres'].apply(lambda x : x.split(','))
data2['original_language'] = 'te'
data2['original_language'] = data2['original_language'].apply(lambda x: [x])

In [13]:
data2['overview'] = data2['overview'].apply(lambda x : x.split(','))

In [14]:
data2.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language
0,Ek Niranjan,https://www.themoviedb.org/movie/38220?languag...,38220,"[Romance, Action, Drama]","[A, bounty, hunter, searches, for, the, family...","[Prabhas, KanganaRanaut, SonuSood]",[[PuriJagannadh]],[te]
1,Bāhubali: The Beginning,https://www.themoviedb.org/movie/256040?langua...,256040,"[Action, Adventure, Drama, Fantasy]","[The, young, Shivudu, is, left, as, a, foundli...","[Prabhas, RanaDaggubati, TamannaahBhatia]",[[S.S.Rajamouli]],[te]
2,Mirchi,https://www.themoviedb.org/movie/176590?langua...,176590,"[Action, Drama]","[A, man, returns, to, his, country, in, order,...","[Prabhas, AnushkaShetty, RichaGangopadhyay]",[[KoratalaSiva]],[te]
3,Rebel,https://www.themoviedb.org/movie/135870?langua...,135870,[Action],"[After, his, parents, are, brutally, murdered,...","[Prabhas, TamannaahBhatia, DeekshaSeth]",[[RaghavaLawrence]],[te]
4,Mr. Perfect,https://www.themoviedb.org/movie/63376?languag...,63376,"[Romance, Comedy]","[Vicky, breaks, his, engagement, with, Priya, ...","[Prabhas, KajalAgarwal, TaapseePannu]",[[Dasaradh]],[te]


In [ ]:
data2['tags'] = data2['genres']+ data2['overview'] + data2['cast'] + data2['crew']
data2['tags'] = data2['tags']+data2['original_language']

In [17]:
data2['crew'] = data2['crew'].apply(lambda x : [item for sublist in x for item in sublist])

In [18]:
data2.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language
0,Ek Niranjan,https://www.themoviedb.org/movie/38220?languag...,38220,"[Romance, Action, Drama]","[A, bounty, hunter, searches, for, the, family...","[Prabhas, KanganaRanaut, SonuSood]",[PuriJagannadh],[te]
1,Bāhubali: The Beginning,https://www.themoviedb.org/movie/256040?langua...,256040,"[Action, Adventure, Drama, Fantasy]","[The, young, Shivudu, is, left, as, a, foundli...","[Prabhas, RanaDaggubati, TamannaahBhatia]",[S.S.Rajamouli],[te]
2,Mirchi,https://www.themoviedb.org/movie/176590?langua...,176590,"[Action, Drama]","[A, man, returns, to, his, country, in, order,...","[Prabhas, AnushkaShetty, RichaGangopadhyay]",[KoratalaSiva],[te]
3,Rebel,https://www.themoviedb.org/movie/135870?langua...,135870,[Action],"[After, his, parents, are, brutally, murdered,...","[Prabhas, TamannaahBhatia, DeekshaSeth]",[RaghavaLawrence],[te]
4,Mr. Perfect,https://www.themoviedb.org/movie/63376?languag...,63376,"[Romance, Comedy]","[Vicky, breaks, his, engagement, with, Priya, ...","[Prabhas, KajalAgarwal, TaapseePannu]",[Dasaradh],[te]


In [26]:
print(" ".join(data2['tags'].iloc[-200]))

Drama Romance Prabha prepares to meet Nandini  his childhood friend  at a reunion party organised by his father. However  things take a turn when Nisha  a gangster's daughter  falls in love with him. Prabhas KajalAgarwal ShraddhaDas A.Karunakaran te


## Now we will extract bollywood movies data

In [27]:
driver = webdriver.Chrome()
driver.get("https://www.themoviedb.org/list/13650?language=en-US")

In [28]:
soup = BeautifulSoup(driver.page_source,"html.parser")

In [29]:
bollywood_movies = []
import re
import numpy as np

for movie in soup.find('ul',class_="items_wrapper list_results").find_all('li'):
    try:
        title = movie.find('img').get('alt')
        movie_link = "https://www.themoviedb.org/"+movie.find('a').get('href')[1:]
        movie_id = re.split(r'[/?]', movie_link)[4]
        bollywood_movies.append([title,movie_link,movie_id])
    except:
        title = np.nan
        movie_link = np.nan
        movie_id = np.nan
        
import pandas as pd
bollywood_data = pd.DataFrame(bollywood_movies,columns=["title","movie_link","movie_id"])

In [30]:
bollywood_data.head()

,title,movie_link,movie_id
0,2 States,https://www.themoviedb.org/movie/259720?langua...,259720
1,7 Khoon Maaf,https://www.themoviedb.org/movie/56338?languag...,56338
2,A Gentleman,https://www.themoviedb.org/movie/469872?langua...,469872
3,Aarakshan,https://www.themoviedb.org/movie/72152?languag...,72152
4,Aashiqui 2,https://www.themoviedb.org/movie/192558?langua...,192558


In [31]:
def getMovieDetails(movieID):
    details = []
    response1 = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movieID,tmdb.api_key))
    response2 = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movieID,tmdb.api_key))
    resp1_json = response1.json()
    resp2_json = response2.json()
    
    genres = []
    for genre in resp1_json['genres']:
        genres.append(genre['name'])
    genres = ",".join(genres)
        
    overview = resp1_json['overview'].replace(" ",",")
    
    casting = []
    for i in range(0,3):
        if resp2_json['cast'][i]['name']:
            casting.append(resp2_json['cast'][i]['name'].replace(" ",''))
        else:
            break
    
    crew=""
    for role in resp2_json['crew']:
        if role['job'] == 'Director':
            crew = role['name']
            break
    crew = crew.replace(" ","")

    return [genres, overview, casting, crew]


new_columns = bollywood_data['movie_id'].apply(getMovieDetails)
new_columns_df = pd.DataFrame(new_columns.tolist(), columns=['genres', 'overview', 'cast', 'crew'])
bollywood_data = pd.concat([bollywood_data, new_columns_df], axis=1)

In [32]:
bollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew
0,2 States,https://www.themoviedb.org/movie/259720?langua...,259720,"Comedy,Romance,Drama","A,man,and,a,woman,come,from,two,very,different...","[AliaBhatt, ArjunKapoor, AmritaSingh]",AbhishekVarman
1,7 Khoon Maaf,https://www.themoviedb.org/movie/56338?languag...,56338,"Drama,Mystery,Thriller","Susanna,is,hungry,for,love,and,will,go,to,any,...","[PriyankaChopraJonas, VivaanShah, KonkonaSenSh...",VishalBhardwaj
2,A Gentleman,https://www.themoviedb.org/movie/469872?langua...,469872,"Action,Comedy","Gaurav,dreams,of,settling,down,with,Kavya,,the...","[SidharthMalhotra, JacquelineFernandez, Suniel...",KrishnaD.K.
3,Aarakshan,https://www.themoviedb.org/movie/72152?languag...,72152,"Drama,Thriller","The,decision,by,India's,supreme,court,to,estab...","[AmitabhBachchan, SaifAliKhan, DeepikaPadukone]",PrakashJha
4,Aashiqui 2,https://www.themoviedb.org/movie/192558?langua...,192558,"Drama,Romance","A,young,woman,meets,a,failing,musician,who,lau...","[AdityaRoyKapur, ShraddhaKapoor, ShaadRandhawa]",MohitSuri


In [33]:
len(bollywood_data)
bollywood_data['original_language'] = 'hi'

In [34]:
bollywood_data['original_language'] = bollywood_data['original_language'].apply(lambda x: [x])
bollywood_data['overview'] = bollywood_data['overview'].apply(lambda x : x.split(','))
bollywood_data['genres'] = bollywood_data['genres'].apply(lambda x : x.split(','))
bollywood_data['crew'] = bollywood_data['crew'].apply(lambda x : [x])

In [35]:
bollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language
0,2 States,https://www.themoviedb.org/movie/259720?langua...,259720,"[Comedy, Romance, Drama]","[A, man, and, a, woman, come, from, two, very,...","[AliaBhatt, ArjunKapoor, AmritaSingh]",[AbhishekVarman],[hi]
1,7 Khoon Maaf,https://www.themoviedb.org/movie/56338?languag...,56338,"[Drama, Mystery, Thriller]","[Susanna, is, hungry, for, love, and, will, go...","[PriyankaChopraJonas, VivaanShah, KonkonaSenSh...",[VishalBhardwaj],[hi]
2,A Gentleman,https://www.themoviedb.org/movie/469872?langua...,469872,"[Action, Comedy]","[Gaurav, dreams, of, settling, down, with, Kav...","[SidharthMalhotra, JacquelineFernandez, Suniel...",[KrishnaD.K.],[hi]
3,Aarakshan,https://www.themoviedb.org/movie/72152?languag...,72152,"[Drama, Thriller]","[The, decision, by, India's, supreme, court, t...","[AmitabhBachchan, SaifAliKhan, DeepikaPadukone]",[PrakashJha],[hi]
4,Aashiqui 2,https://www.themoviedb.org/movie/192558?langua...,192558,"[Drama, Romance]","[A, young, woman, meets, a, failing, musician,...","[AdityaRoyKapur, ShraddhaKapoor, ShaadRandhawa]",[MohitSuri],[hi]


In [39]:
bollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language,tags
0,2 States,https://www.themoviedb.org/movie/259720?langua...,259720,"[Comedy, Romance, Drama]","[A, man, and, a, woman, come, from, two, very,...","[AliaBhatt, ArjunKapoor, AmritaSingh]",[AbhishekVarman],[hi],"[Comedy, Romance, Drama, A, man, and, a, woman..."
1,7 Khoon Maaf,https://www.themoviedb.org/movie/56338?languag...,56338,"[Drama, Mystery, Thriller]","[Susanna, is, hungry, for, love, and, will, go...","[PriyankaChopraJonas, VivaanShah, KonkonaSenSh...",[VishalBhardwaj],[hi],"[Drama, Mystery, Thriller, Susanna, is, hungry..."
2,A Gentleman,https://www.themoviedb.org/movie/469872?langua...,469872,"[Action, Comedy]","[Gaurav, dreams, of, settling, down, with, Kav...","[SidharthMalhotra, JacquelineFernandez, Suniel...",[KrishnaD.K.],[hi],"[Action, Comedy, Gaurav, dreams, of, settling,..."
3,Aarakshan,https://www.themoviedb.org/movie/72152?languag...,72152,"[Drama, Thriller]","[The, decision, by, India's, supreme, court, t...","[AmitabhBachchan, SaifAliKhan, DeepikaPadukone]",[PrakashJha],[hi],"[Drama, Thriller, The, decision, by, India's, ..."
4,Aashiqui 2,https://www.themoviedb.org/movie/192558?langua...,192558,"[Drama, Romance]","[A, young, woman, meets, a, failing, musician,...","[AdityaRoyKapur, ShraddhaKapoor, ShaadRandhawa]",[MohitSuri],[hi],"[Drama, Romance, A, young, woman, meets, a, fa..."


In [42]:
" ".join(bollywood_data['tags'].iloc[-100])

"Drama Romance A journey of three restless young people who junk the society's syllabus for finding love and decide to follow their heart. SushantSinghRajput ParineetiChopra VaaniKapoor ManeeshSharma hi"

## Now we will collect tamil movies collection from tmdb

In [43]:
driver = webdriver.Chrome()
driver.get("https://www.themoviedb.org/list/129716?language=en-US")

In [44]:
soup = BeautifulSoup(driver.page_source,"html.parser")

kollywood_movies = []
import re
import numpy as np

for movie in soup.find('ul',class_="items_wrapper list_results").find_all('li'):
    try:
        title = movie.find('img').get('alt')
        movie_link = "https://www.themoviedb.org/"+movie.find('a').get('href')[1:]
        movie_id = re.split(r'[/?]', movie_link)[4]
        kollywood_movies.append([title,movie_link,movie_id])
    except:
        title = np.nan
        movie_link = np.nan
        movie_id = np.nan
        
import pandas as pd
kollywood_data = pd.DataFrame(kollywood_movies,columns=["title","movie_link","movie_id"])

In [45]:
kollywood_data.head()

,title,movie_link,movie_id
0,Darbar,https://www.themoviedb.org/movie/594139?langua...,594139
1,Psycho,https://www.themoviedb.org/movie/642889?langua...,642889
2,Maayanadhi,https://www.themoviedb.org/movie/669117?langua...,669117
3,Naadodigal 2,https://www.themoviedb.org/movie/504409?langua...,504409
4,Dagaalty,https://www.themoviedb.org/movie/607138?langua...,607138


In [46]:
def getMovieDetails(movieID):
    details = []
    response1 = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movieID,tmdb.api_key))
    response2 = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movieID,tmdb.api_key))
    resp1_json = response1.json()
    resp2_json = response2.json()
    
    genres = []
    for genre in resp1_json['genres']:
        genres.append(genre['name'])
    genres = ",".join(genres)
        
    overview = resp1_json['overview'].replace(" ",",")
    
    casting = []
    for i in range(0,3):
        if resp2_json['cast'][i]['name']:
            casting.append(resp2_json['cast'][i]['name'].replace(" ",''))
        else:
            break
    
    crew=""
    for role in resp2_json['crew']:
        if role['job'] == 'Director':
            crew = role['name']
            break
    crew = crew.replace(" ","")

    return [genres, overview, casting, crew]


new_columns = kollywood_data['movie_id'].apply(getMovieDetails)
new_columns_df = pd.DataFrame(new_columns.tolist(), columns=['genres', 'overview', 'cast', 'crew'])
kollywood_data = pd.concat([kollywood_data, new_columns_df], axis=1)

In [47]:
kollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew
0,Darbar,https://www.themoviedb.org/movie/594139?langua...,594139,"Action,Drama,Thriller","Aaditya,,the,commissioner,of,Mumbai,Police,,se...","[Rajinikanth, Nayanthara, SunielShetty]",A.R.Murugadoss
1,Psycho,https://www.themoviedb.org/movie/642889?langua...,642889,"Thriller,Drama","Based,on,the,Buddhist,tale,of,Angulimala,,a,dr...","[RajkumarPitchumani, AditiRaoHydari, Udhayanid...",Mysskin
2,Maayanadhi,https://www.themoviedb.org/movie/669117?langua...,669117,"Drama,Family","A,doting,father,and,her,daughter,,who,aspires,...","[Venba, AdukalamNaren, AbiSaravanan]",AshokThiagarajan
3,Naadodigal 2,https://www.themoviedb.org/movie/504409?langua...,504409,"Comedy,Drama","A,set,of,befriends,jointly,saving,a,lovers,aga...","[M.Sasikumar, Anjali, AthulyaRavi]",Samuthirakani
4,Dagaalty,https://www.themoviedb.org/movie/607138?langua...,607138,"Comedy,Action","A,conman,tries,to,save,an,innocent,girl,from,t...","[Santhanam, RittikaSen, YogiBabu]",VijayAnand


In [48]:
kollywood_data['original_language'] = 'ta'
kollywood_data['original_language'] = kollywood_data['original_language'].apply(lambda x: [x])
kollywood_data['overview'] = kollywood_data['overview'].apply(lambda x : x.split(','))
kollywood_data['genres'] = kollywood_data['genres'].apply(lambda x : x.split(','))
kollywood_data['crew'] = kollywood_data['crew'].apply(lambda x : [x])
kollywood_data['tags'] = kollywood_data['genres']+ kollywood_data['overview'] + kollywood_data['cast'] + kollywood_data['crew'] + kollywood_data['original_language']


In [49]:
kollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language,tags
0,Darbar,https://www.themoviedb.org/movie/594139?langua...,594139,"[Action, Drama, Thriller]","[Aaditya, , the, commissioner, of, Mumbai, Pol...","[Rajinikanth, Nayanthara, SunielShetty]",[A.R.Murugadoss],[ta],"[Action, Drama, Thriller, Aaditya, , the, comm..."
1,Psycho,https://www.themoviedb.org/movie/642889?langua...,642889,"[Thriller, Drama]","[Based, on, the, Buddhist, tale, of, Angulimal...","[RajkumarPitchumani, AditiRaoHydari, Udhayanid...",[Mysskin],[ta],"[Thriller, Drama, Based, on, the, Buddhist, ta..."
2,Maayanadhi,https://www.themoviedb.org/movie/669117?langua...,669117,"[Drama, Family]","[A, doting, father, and, her, daughter, , who,...","[Venba, AdukalamNaren, AbiSaravanan]",[AshokThiagarajan],[ta],"[Drama, Family, A, doting, father, and, her, d..."
3,Naadodigal 2,https://www.themoviedb.org/movie/504409?langua...,504409,"[Comedy, Drama]","[A, set, of, befriends, jointly, saving, a, lo...","[M.Sasikumar, Anjali, AthulyaRavi]",[Samuthirakani],[ta],"[Comedy, Drama, A, set, of, befriends, jointly..."
4,Dagaalty,https://www.themoviedb.org/movie/607138?langua...,607138,"[Comedy, Action]","[A, conman, tries, to, save, an, innocent, gir...","[Santhanam, RittikaSen, YogiBabu]",[VijayAnand],[ta],"[Comedy, Action, A, conman, tries, to, save, a..."


In [54]:
" ".join(kollywood_data['tags'].iloc[20])

'Comedy Fantasy A TV reporter  who has been gathering evidence to expose a self-styled godman for years  gets help from Goddess Mookuthi Amman. How does he utilise the situation to bring down the godman before the public? Nayanthara BalajiPatturaj Urvashi BalajiPatturaj ta'

## Now we will collect Malayalam movies from TMDb

In [55]:
driver = webdriver.Chrome()
driver.get("https://www.themoviedb.org/list/6565?language=ar-AE")

In [56]:
soup = BeautifulSoup(driver.page_source,"html.parser")

mollywood_movies = []
import re
import numpy as np

for movie in soup.find('ul',class_="items_wrapper list_results").find_all('li'):
    try:
        title = movie.find('img').get('alt')
        movie_link = "https://www.themoviedb.org/"+movie.find('a').get('href')[1:]
        movie_id = re.split(r'[/?]', movie_link)[4]
        mollywood_movies.append([title,movie_link,movie_id])
    except:
        title = np.nan
        movie_link = np.nan
        movie_id = np.nan
        
import pandas as pd
mollywood_data = pd.DataFrame(mollywood_movies,columns=["title","movie_link","movie_id"])

In [57]:
mollywood_data.head()

,title,movie_link,movie_id
0,ലൈലാ ഓ ലൈലാ,https://www.themoviedb.org/movie/338314?langua...,338314
1,ധ്രുവം,https://www.themoviedb.org/movie/78681?languag...,78681
2,രാജമാണിക്യം,https://www.themoviedb.org/movie/152359?langua...,152359
3,ഡാഡി കൂൾ,https://www.themoviedb.org/movie/202717?langua...,202717
4,ഹിറ്റ്ലർ,https://www.themoviedb.org/movie/237735?langua...,237735


In [58]:
def getMovieDetails(movieID):
    details = []
    response1 = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movieID,tmdb.api_key))
    response2 = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movieID,tmdb.api_key))
    resp1_json = response1.json()
    resp2_json = response2.json()
    
    genres = []
    for genre in resp1_json['genres']:
        genres.append(genre['name'])
    genres = ",".join(genres)
        
    overview = resp1_json['overview'].replace(" ",",")
    
    casting = []
    for i in range(0,3):
        if resp2_json['cast'][i]['name']:
            casting.append(resp2_json['cast'][i]['name'].replace(" ",''))
        else:
            break
    
    crew=""
    for role in resp2_json['crew']:
        if role['job'] == 'Director':
            crew = role['name']
            break
    crew = crew.replace(" ","")

    return [genres, overview, casting, crew]


new_columns = mollywood_data['movie_id'].apply(getMovieDetails)
new_columns_df = pd.DataFrame(new_columns.tolist(), columns=['genres', 'overview', 'cast', 'crew'])
mollywood_data = pd.concat([mollywood_data, new_columns_df], axis=1)

In [59]:
mollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew
0,ലൈലാ ഓ ലൈലാ,https://www.themoviedb.org/movie/338314?langua...,338314,"Action,Thriller","The,story,of,the,film,revolves,around,Intellig...","[Mohanlal, AmalaPaul, Sathyaraj]",Joshiy
1,ധ്രുവം,https://www.themoviedb.org/movie/78681?languag...,78681,"Action,Drama","Veerasimha,Mannadiar,gets,killed,by,Hyder,Mara...","[Mammootty, GautamiTadimalla, Jayaram]",Joshiy
2,രാജമാണിക്യം,https://www.themoviedb.org/movie/152359?langua...,152359,"Action,Comedy","Bellary,Raja,is,disowned,by,his,stepfather,in,...","[Mammootty, Rahman, ManojKJayan]",AnwarRasheed
3,ഡാഡി കൂൾ,https://www.themoviedb.org/movie/202717?langua...,202717,"Comedy,Action","The,story,line,of,the,film,is,the,escapades,of...","[Mammootty, Dhananjay, RichaPallod]",AashiqAbu
4,ഹിറ്റ്ലർ,https://www.themoviedb.org/movie/237735?langua...,237735,Drama,"Madhavan,,an,eligible,bachelor,,is,very,protec...","[Mammootty, Mukesh, Shobana]",Siddique


In [60]:
mollywood_data['original_language'] = 'ma'
mollywood_data['original_language'] = mollywood_data['original_language'].apply(lambda x: [x])
mollywood_data['overview'] = mollywood_data['overview'].apply(lambda x : x.split(','))
mollywood_data['genres'] = mollywood_data['genres'].apply(lambda x : x.split(','))
mollywood_data['crew'] = mollywood_data['crew'].apply(lambda x : [x])
mollywood_data['tags'] = mollywood_data['genres']+ mollywood_data['overview'] + mollywood_data['cast'] + mollywood_data['crew'] + mollywood_data['original_language']


In [61]:
mollywood_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language,tags
0,ലൈലാ ഓ ലൈലാ,https://www.themoviedb.org/movie/338314?langua...,338314,"[Action, Thriller]","[The, story, of, the, film, revolves, around, ...","[Mohanlal, AmalaPaul, Sathyaraj]",[Joshiy],[ma],"[Action, Thriller, The, story, of, the, film, ..."
1,ധ്രുവം,https://www.themoviedb.org/movie/78681?languag...,78681,"[Action, Drama]","[Veerasimha, Mannadiar, gets, killed, by, Hyde...","[Mammootty, GautamiTadimalla, Jayaram]",[Joshiy],[ma],"[Action, Drama, Veerasimha, Mannadiar, gets, k..."
2,രാജമാണിക്യം,https://www.themoviedb.org/movie/152359?langua...,152359,"[Action, Comedy]","[Bellary, Raja, is, disowned, by, his, stepfat...","[Mammootty, Rahman, ManojKJayan]",[AnwarRasheed],[ma],"[Action, Comedy, Bellary, Raja, is, disowned, ..."
3,ഡാഡി കൂൾ,https://www.themoviedb.org/movie/202717?langua...,202717,"[Comedy, Action]","[The, story, line, of, the, film, is, the, esc...","[Mammootty, Dhananjay, RichaPallod]",[AashiqAbu],[ma],"[Comedy, Action, The, story, line, of, the, fi..."
4,ഹിറ്റ്ലർ,https://www.themoviedb.org/movie/237735?langua...,237735,[Drama],"[Madhavan, , an, eligible, bachelor, , is, ver...","[Mammootty, Mukesh, Shobana]",[Siddique],[ma],"[Drama, Madhavan, , an, eligible, bachelor, , ..."


In [63]:
" ".join(kollywood_data['tags'].iloc[-10])

'Adventure Drama A young man travels across India and on his way he learns about the issues connected to nation-wide politics which urges him to do something about it. Jiiva NatashaSingh SunnyWayne RajuMurugan ta'

## We will merge all languages movies into a single file

In [64]:
indian_movies_data = pd.concat([data2,bollywood_data,kollywood_data,mollywood_data])

In [65]:
indian_movies_data.head()

,title,movie_link,movie_id,genres,overview,cast,crew,original_language,tags
0,Ek Niranjan,https://www.themoviedb.org/movie/38220?languag...,38220,"[Romance, Action, Drama]","[A, bounty, hunter, searches, for, the, family...","[Prabhas, KanganaRanaut, SonuSood]",[PuriJagannadh],[te],"[Romance, Action, Drama, A, bounty, hunter, se..."
1,Bāhubali: The Beginning,https://www.themoviedb.org/movie/256040?langua...,256040,"[Action, Adventure, Drama, Fantasy]","[The, young, Shivudu, is, left, as, a, foundli...","[Prabhas, RanaDaggubati, TamannaahBhatia]",[S.S.Rajamouli],[te],"[Action, Adventure, Drama, Fantasy, The, young..."
2,Mirchi,https://www.themoviedb.org/movie/176590?langua...,176590,"[Action, Drama]","[A, man, returns, to, his, country, in, order,...","[Prabhas, AnushkaShetty, RichaGangopadhyay]",[KoratalaSiva],[te],"[Action, Drama, A, man, returns, to, his, coun..."
3,Rebel,https://www.themoviedb.org/movie/135870?langua...,135870,[Action],"[After, his, parents, are, brutally, murdered,...","[Prabhas, TamannaahBhatia, DeekshaSeth]",[RaghavaLawrence],[te],"[Action, After, his, parents, are, brutally, m..."
4,Mr. Perfect,https://www.themoviedb.org/movie/63376?languag...,63376,"[Romance, Comedy]","[Vicky, breaks, his, engagement, with, Priya, ...","[Prabhas, KajalAgarwal, TaapseePannu]",[Dasaradh],[te],"[Romance, Comedy, Vicky, breaks, his, engageme..."


In [66]:
len(indian_movies_data)

523

In [67]:
indian_movies = indian_movies_data[["movie_id","title","tags"]]

In [68]:
indian_movies.head()

,movie_id,title,tags
0,38220,Ek Niranjan,"[Romance, Action, Drama, A, bounty, hunter, se..."
1,256040,Bāhubali: The Beginning,"[Action, Adventure, Drama, Fantasy, The, young..."
2,176590,Mirchi,"[Action, Drama, A, man, returns, to, his, coun..."
3,135870,Rebel,"[Action, After, his, parents, are, brutally, m..."
4,63376,Mr. Perfect,"[Romance, Comedy, Vicky, breaks, his, engageme..."


In [72]:
indian_movies['tags'] = indian_movies['tags'].apply(lambda x  : " ".join(x).lower())

C:\Users\Lokesh\AppData\Local\Temp\ipykernel_18332\2812176130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indian_movies['tags'] = indian_movies['tags'].apply(lambda x  : " ".join(x).lower())


In [73]:
indian_movies.head()

,movie_id,title,tags
0,38220,Ek Niranjan,romance action drama a bounty hunter searches ...
1,256040,Bāhubali: The Beginning,action adventure drama fantasy the young shivu...
2,176590,Mirchi,action drama a man returns to his country in o...
3,135870,Rebel,action after his parents are brutally murdered...
4,63376,Mr. Perfect,romance comedy vicky breaks his engagement wit...


In [ ]:
indian_movies.to_csv('IndianM',index=False)